# Sine wave Regression Using Reptile

In [2]:
# Import required libraries

import tensorflow.compat.v1 as tf
import numpy as np

In [3]:
# Generate Data Points

def generate_sample_points(n):
  num_point = 150
  amplitude = np.random.uniform(low=0.1, high=6.0)
  phase = np.random.uniform(low=0, high=np.pi)
  x = np.linspace(-6,6,num_point)
  y= amplitude * np.sin(x + phase)
  sample = np.random.choice(np.arange(num_point), size=n)
  return (x[sample],y[sample])

In [4]:
x, y = generate_sample_points(5)
print(x)

[ 1.81208054  4.06711409 -0.36241611 -1.73154362 -5.3557047 ]


In [5]:
print(y)

[-3.96484734  1.88181819  2.91404941  3.33348459 -1.88883152]


In [6]:
# We will build two layerd neural network.
# Reset tensorflow graph

tf.compat.v1.reset_default_graph()

In [7]:
# Parameters
numHidden = 64
numClasses = 1
numFeature = 1

In [8]:
# Define placeholder for X and Y
tf.compat.v1.disable_eager_execution()
x = tf.compat.v1.placeholder(tf.float32, shape=[None, numFeature])
y = tf.compat.v1.placeholder(tf.float32, shape=[None, numClasses])

In [9]:
# Initialize model parameters
w1 = tf.Variable(tf.random_uniform([numFeature, numHidden]))
b1 = tf.Variable(tf.random_uniform([numHidden]))

w2 = tf.Variable(tf.random_uniform([numHidden, numClasses]))
b2 = tf.Variable(tf.random_uniform([numClasses]))

In [10]:
# Feedforward operation

#layer 1
z1 = tf.matmul(x, w1) + b1
a1 = tf.nn.tanh(z1)

#output layer
z2 = tf.matmul(a1, w2) + b2
Yhat = tf.nn.tanh(z2)

In [12]:
#Loss function

lossFunction = tf.reduce_mean(tf.square(Yhat - y))

In [13]:
# Optimizer
optimizer = tf.train.AdamOptimizer(1e-2).minimize(lossFunction)

In [14]:
# Initialize tensorflow variable
init = tf.global_variables_initializer()

In [15]:
# Neural Net with Reptile

# Initialize variables
numEpochs = 100
numSamples = 50  
numTasks = 2
numIterations = 10
miniBatch = 10

In [18]:
#start the tensorflow session
with tf.Session() as s:
    
    s.run(init)
    
    for e in range(numEpochs):
        for task in range(numTasks):

            #get the initial parameter
            old_w1, old_b1, old_w2, old_b2 = s.run([w1, b1, w2, b2,])

            #sample x and y
            x_sample, y_sample = generate_sample_points(numSamples)


            # Perform optimization on the task
            for k in range(numIterations):
                for i in range(0, numSamples, miniBatch):
                    x_minibatch = x_sample[i:i+miniBatch]
                    y_minibatch = y_sample[i:i+miniBatch]

                    train = s.run(optimizer, feed_dict={x: x_minibatch.reshape(miniBatch,1), 
                                                           y: y_minibatch.reshape(miniBatch,1)})

            #get the updated model parameter
            new_w1, new_b1, new_w2, new_b2 = s.run([w1, b1, w2, b2])

            # Perform meta update
            epsilon = 0.1

            updated_w1 = old_w1 + epsilon * (new_w1 - old_w1) 
            updated_b1 = old_b1 + epsilon * (new_b1 - old_b1) 

            updated_w2 = old_w2 + epsilon * (new_w2 - old_w2) 
            updated_b2 = old_b2 + epsilon * (new_b2 - old_b2) 


            #update the model parameter with new parameters
            w1.load(updated_w1, s)
            b1.load(updated_b1, s)

            w2.load(updated_w2, s)
            b2.load(updated_b2, s)

        if e%10 == 0:
            loss = s.run(lossFunction, feed_dict={x: x_sample.reshape(numSamples,1), y: y_sample.reshape(numSamples,1)})

            print("Epoch {}: Loss {}\n".format(e,loss))             
            print('Updated Model Parameter Theta\n')
            print('Sampling Next Batch of Tasks \n')
            print('---------------------------------\n')


Instructions for updating:
Prefer Variable.assign which has equivalent behavior in 2.X.
Epoch 0: Loss 2.201439619064331

Updated Model Parameter Theta

Sampling Next Batch of Tasks 

---------------------------------

Epoch 10: Loss 1.1509068012237549

Updated Model Parameter Theta

Sampling Next Batch of Tasks 

---------------------------------

Epoch 20: Loss 6.679961681365967

Updated Model Parameter Theta

Sampling Next Batch of Tasks 

---------------------------------

Epoch 30: Loss 5.142261981964111

Updated Model Parameter Theta

Sampling Next Batch of Tasks 

---------------------------------

Epoch 40: Loss 3.879302978515625

Updated Model Parameter Theta

Sampling Next Batch of Tasks 

---------------------------------

Epoch 50: Loss 0.950084388256073

Updated Model Parameter Theta

Sampling Next Batch of Tasks 

---------------------------------

Epoch 60: Loss 1.839415431022644

Updated Model Parameter Theta

Sampling Next Batch of Tasks 

------------------------------